# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [1]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.dot(data_set, data_set.T) ** 2
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [89]:
def choose_set_for_label(data_set, label, labels):
    # your code here
    '''
    set_for_label = []
    for i in range(data_set.shape[1] - 1):
        temp_data = data_set[labels!= i]
        temp_labels = labels[labels!= i]
        train_data_set, test_data_set, train_labels, test_labels = train_test_split(
        temp_data, temp_labels, test_size=0.2, random_state=15)
        train_labels[train_labels<1] = -1
        test_labels[test_labels<1] = -1
        set_for_label.append([train_data_set, test_data_set, train_labels, test_labels])'''
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
        data_set, labels, test_size=0.2, random_state=15)
    train_labels[train_labels != label] = -1
    test_labels[test_labels!= label] = -1
    
    train_labels[train_labels == label] = 1
    test_labels[test_labels == label] = 1
    
    return  train_data_set, test_data_set, train_labels, test_labels

In [90]:
def get_labels_count(data_set):
    return len(data_set)

In [91]:
#Use the code that we have implemented earlier:

In [92]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b), kktsolver='ldl', options={'kktreg':1e-9})

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [93]:
# modify this part 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import cvxopt

iris = load_iris()
labels = iris.target
data_set = iris.data
predictions = []
tests = []
'''
# for 3 classes one to one approach
data_sets = choose_set_for_label(data_set, labels)
objects_count = get_labels_count(data_sets[0][0])
predictions = []
true_labels = []

for i in range(len(data_sets)):
    train_data_set, test_data_set, train_labels, test_labels = data_sets[i]
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predictions.append(predicted)
    true_labels.append(test_labels)
    #predicted = list(predicted.astype(int))

#voting part
votes = [[0] * len(predictions[0]) for _ in range(len(predictions)) ]
final_prediction = [0]*len(predictions[0])

# 1vs2
for i in range(len(predictions[0])):
    if predictions[0][i] == -1:
        votes[1][i] += 1
    else:
        votes[2][i] += 1  
# 0vs2
for i in range(len(predictions[1])):
    if predictions[1][i] == -1:
        votes[0][i] += 1
    else:
        votes[2][i] += 1  
# 0vs1
for i in range(len(predictions[2])):
    if predictions[2][i] == -1:
        votes[0][i] += 1
    else:
        votes[1][i] += 1
for i in range(len(votes[0])):
    max_idx = 0
    max_freq = 1
    for j in range(len(votes)):
        if votes[j][i] >= max_freq and j == 1:
            max_freq = votes[j][i]
            max_idx = j
        
        final_prediction[i] = max_idx
        

classes = np.unique(np.array(final_prediction))
if len(classes) == 2:
    class_neg = classes[1]
    for i in range(len(predictions[0])):
        if final_prediction[i] == class_neg:
            final_prediction[i] = 1
        else:
            final_prediction[i] = -1
'''

for label in np.unique(labels):
    
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, label, labels)
    objects_count = get_labels_count(train_labels)
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predictions.append(predicted)
    tests.append(test_labels)


from sklearn.metrics import accuracy_score
for i in range(len(predictions)):
    print(accuracy_score(list(predictions[i].astype(int)), tests[i]))


   

     pcost       dcost       gap    pres   dres
 0:  1.1726e+02 -1.7992e+03  4e+03  2e-01  4e-10
 1:  7.7967e+01 -1.6973e+02  3e+02  5e-03  2e-10
 2:  1.0169e+01 -2.2323e+01  3e+01  1e-09  1e-10
 3: -4.9607e-01 -4.9938e+00  4e+00  8e-10  5e-11
 4: -1.4344e+00 -2.5218e+00  1e+00  8e-10  2e-11
 5: -1.6897e+00 -2.1674e+00  5e-01  2e-10  5e-11
 6: -1.8078e+00 -2.0073e+00  2e-01  2e-10  4e-11
 7: -1.8464e+00 -1.9763e+00  1e-01  3e-10  3e-11
 8: -1.8825e+00 -1.9020e+00  2e-02  9e-10  2e-11
 9: -1.8902e+00 -1.8906e+00  4e-04  1e-09  5e-12
10: -1.8904e+00 -1.8904e+00  5e-06  1e-09  3e-13
11: -1.8904e+00 -1.8904e+00  5e-08  1e-09  9e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.1410e+02 -2.4247e+03  5e+03  2e-01  5e-10
 1:  9.0959e+01 -2.2448e+02  4e+02  6e-03  3e-10
 2:  1.3024e+01 -2.6284e+01  4e+01  4e-05  1e-10
 3: -1.1088e-01 -5.5940e+00  5e+00  8e-10  6e-11
 4: -1.3519e+00 -2.4130e+00  1e+00  9e-10  2e-11
 5: -1.5743e+00 -2.0232e+00  4e-01  2e-10  3e-1